In [2]:
import json
from camdkit.timing_types import Timecode
print(json.dumps(Timecode.make_json_schema(), indent=2))


{
  "type": "object",
  "properties": {
    "hours": {
      "type": "integer",
      "maximum": 23,
      "minimum": 0
    },
    "minutes": {
      "type": "integer",
      "maximum": 59,
      "minimum": 0
    },
    "seconds": {
      "type": "integer",
      "maximum": 59,
      "minimum": 0
    },
    "frames": {
      "type": "integer",
      "maximum": 119,
      "minimum": 0
    },
    "format": {
      "type": "object",
      "properties": {
        "frameRate": {
          "type": "object",
          "properties": {
            "num": {
              "type": "integer",
              "maximum": 2147483647,
              "minimum": 1
            },
            "denom": {
              "type": "integer",
              "maximum": 4294967295,
              "minimum": 1
            }
          },
          "required": [
            "num",
            "denom"
          ],
          "additionalProperties": false
        },
        "subFrame": {
          "type": "integer",
         

In [6]:
def foo():
    """This is a multline doc string
    with three lines
    worth of stuff
    """
    print('stop here')
foo()


KeyboardInterrupt: 